# INFO-F-208: Project 2
** *Name:* ** Théo Verhelst  
** *Matricule:* ** 000400807  
** *Date:* ** 11/4/2016

## Background and objectives
Alignment of amino acids sequences has been explained in the introduction of the first project. In certain circumstances, we can have sequences known to belong to a specific protein domain, and we may want to align them in order to trace the evolution pattern of a specific domain across multiple species.

In this case, a BLOSUM matrix specialised for that domain would be better than a generic one, because the matrix values would be statistically suited to align that domain.

This is the point of this project: construct a BLOSUM matrix suited to align a specific domain. We will use the BLOCKS database to get aligned short sequences of that domain. These alignments will be used to compute statistical information about the substitutions occuring in that domain, and subsequently to make the BLOSUM matrix.

## Code
The code of this project is mainly functionnal and imperative. Since the algorithm of creation of BLOSUM matrix is clearly divided into successive steps, object-oriented programming would not be the best choice.

In [1]:
from os.path import splitext
from math import log2
from statistics import mean

amino_acids = "ARNDCEQGHILKMFPSTWYV"
# This dict speeds up the translation amino acid => index for amino_acids
# (at least in comparaison with amino_acid.index())
amino_acids_indices = dict((key, i) for i, key in enumerate(amino_acids))
gap_char = "X"

The variable `gap_char` represents the character used to fill gaps in the alignments given in the BLOCKS database. Occurences of this character are discarded from the frequencies matrix.

### The matrix class
In this project, we have matrices that store a value for each pair of amino acids. So it may be convenient to be able to access data right from the pair of amino acid, and hide the translation from characters to list indices in a class. This is the intent of this class.

In [2]:
class CharMatrix:
    """Represents a triangular matrix indexed by a finite set of characters.
    It is suited to store values for relations between amino acids.
    """
    
    def __init__(self, keys, value):
        """Constructs the matrix.
        
        Parameters:
            keys: an iterable of characters, it will be the set of acceptable
                keys for indexing
            value: the initial value to put in the cells.
        """
        self.keys = "".join(list(keys))
        self.indices = dict((key, i) for i, key in enumerate(keys))
        self.matrix = [[value] * len(self.keys) for i in range(len(self.keys))]
    
    def __getitem__(self, key):
        return self.matrix[amino_acids_indices[key[0]]][amino_acids_indices[key[1]]]
            
    def __setitem__(self, key, value):
        self.matrix[self.indices[key[0]]][self.indices[key[1]]] = value
    
    def __str__(self):
        """Converts the matrix to a string."""
        
        cell_width = max(max(len(str(c)) for c in line) for line in self.matrix) + 1
        res = ""
        
        for i in range(len(self.matrix)):
            res += str(self.keys[i]).rjust(cell_width)
            for j in range(len(self.matrix[i])):
                if i >= j:
                    res += str(self.matrix[j][i]).rjust(cell_width) + " "
            res += "\n"
        res += (" " * cell_width) + " ".join([c.rjust(cell_width) for c in self.keys]) + "\n"
        return res

### Splitting domains in BLOCKS
This function simply reads a fasta file containing all BLOCKS of a domain, and create a new file for each BLOCK. It then returns the names of the created files.

In [3]:
def split_domain(domain_filename, number_blocks):
    """Reads a .fasta file containing blocks of a domain, and writes the blocks
    in separate files. The blocks files have the same name as the domain file,
    except that a letter is appended just before the extension.
    
    The domain file must be formatted as explained in the statement of the
    project, thus copied from the BLOCKS database. Although the number of blocks
    could be detected from the first lines of the file, asking the user for this
    information make the code much clearer.
    
    Parameters:
        domain_filename: the filename of the domain file
        number_blocks: the number of differents blocks contained in the file
    
    Return value: the filenames of the created files
    """
    
    # Create the filenames of the block files
    root, extension = splitext(domain_filename)
    block_filenames = [root + "-" + chr(ord("A") + i) + extension \
            for i in range(number_blocks)]
    
    # Open the block files
    block_files = [open(filename, "w") for filename in block_filenames]
    
    try:
        with open(domain_filename) as domain_file:
            for i, line in enumerate(domain_file):
                # local_index is 0 if the line is an identifier line,
                # or (1 + the current block) otherwise
                local_index = i % (number_blocks + 1)
                if local_index != 0:
                    block_files[local_index - 1].write(line)
    finally:
        for block_file in block_files:
            block_file.close()
    
    return block_filenames

### Identity rate
This functions counts the number of identical characters that have the same position in two strings. Note that if a string is longer than the other, trailing characters from the longest string will be discarded from the count.

In [4]:
def identity(string_a, string_b):
    """Computes the identity between two strings, in the range [0, 1]."""
    
    count = sum(1 if a == b else 0 for a, b in zip(string_a, string_b))
    return count / min(len(string_a), len(string_b))

### Clustering condition
This function checks whether a sequence can be accepted in a cluster. Because this decision determines the topology of the clustering, it is more convenient to write it in a separate function.

In [5]:
def is_accepted_in_cluster(sequence, cluster, required_identity):
    """Checks if a sequence can be accepted in a cluster.
    Although the test is rather simple, we could think about more complicated
    criterion, this is why it is a separate function."""
    
    for other_sequence in cluster:
        if identity(other_sequence, sequence) >= required_identity:
            return True
    return False

### Clustering
This functions opens and reads a BLOCK file, and splits all the sequences in clusters, according to the given required identity within a cluster.

In [6]:
def make_clusters(block_filename, required_identity):
    """Creates a list of clusters from the block file, and based on the given
    required identity. A cluster is represented by a list of strings.
    """
    
    clusters = []
    
    with open(block_filename) as file:
        for sequence in file:
            sequence = sequence.strip()
            found = False
            
            # Search for a cluster that can contains the current sequence
            for cluster in clusters:
                if is_accepted_in_cluster(sequence, cluster, required_identity):
                    # Add it if the cluster can accept it
                    cluster.append(sequence)
                    found = True
                    break
                    
            if not found:
                # Create a new cluster if there are no acceptable one
                clusters.append([sequence])
                
    return clusters

### Weighted frequencies
This function calculates the matrix  $f_{a,b}$ for a specific block.

The first part counts the frequency of each amino acid for each column, in each cluster. These frequencies are stored in a list of list of dictionaries. Each sub-list of the main list corresponds to a cluster, each element of these sub-lists corresponds to a column. Each dictionnary have amino acids as keys, and the corresponding frequencies as values.

The second part computes the values of $f_{a, b}$ in an efficient way: all frequencies has already been computed and can be accessed in a $O(1)$ time (thanks to the dicts), and amino acids that have a null frequency are not even iterated (since they do not exists in the corresponding dict). 

In [7]:
def weighted_frequencies(clusters):
    """Computes the weighted frequencies of all amino acids substitutions
    between the given clusters, i.e. the matrix f in the slides.
    """
    
    f = CharMatrix(amino_acids, 0)
    clusters_weights = [1 / len(cluster) for cluster in clusters]
    number_columns = len(clusters[0][0])
    number_clusters = len(clusters)
    counts = [[{} for j in range(number_columns)] for i in range(number_clusters)]
    
    for i, cluster in enumerate(clusters):
        for sequence in cluster:
            for c in range(number_columns):
                if sequence[c] != gap_char:
                    counts[i][c][sequence[c]] = counts[i][c].setdefault(
                            sequence[c], 0) + 1
    
    # Iterate over each possible pair of distinct clusters
    for i in range(number_clusters):
        for j in range(i + 1, number_clusters):
            weight = clusters_weights[i] * clusters_weights[j]
            # For each column of amino acid
            for col in range(number_columns):
                for a, frequency_a in counts[i][col].items():
                    for b, frequency_b in counts[j][col].items():
                        if amino_acids_indices[a] < amino_acids_indices[b]:
                            f[a, b] += weight * frequency_a * frequency_b
                        else:
                            f[b, a] += weight * frequency_a * frequency_b
    return f

### Normalized sum
This function computes a matrix whose elements are the mean of the corresponding elements in the given matrices.
This is requested in the statement:
>Les valeurs $f_{a,b}$ sont calculées sur les 4 BLOCK indépendamment.
>Après le $f_{a,b}$ total pour tous les BLOCK ensemble est obtenu en faisant la somme normalisée des ces $f_{a,b}$ par BLOCK.	

In [8]:
def normalized_sum(matrices):
    """Computes the mean of the given matrices."""
    
    keys = matrices[0].keys
    res = CharMatrix(keys, 0)
    
    for a in keys:
        for b in keys[keys.index(a):]:
            res[a, b] = mean(matrix[a, b] for matrix in matrices)
    
    return res

### Occurence probabilities in the evolution pattern
Here we compute the occurrence probabilities of a substitution in the evolution pattern, i.e.
$$q_{a, b}=\frac{f_{a,b}}{\sum_{1\le i\le j}f_{i,j}}$$
in the slides.  
**Note:** the formula in the slides is exactly
$$q_{a, b}=\frac{f_{a,b}}{\sum_{1\le a\le b}f_{a,b}}$$
so I guessed that indices in the sum are different from the indices above the fraction. This leads to a consistent interpretation of the formula.

**Other note:** from there, almost all remaining functions are just translations of formulas into Python code.

In [9]:
def biological_probabilities(f):
    """Computes the probabilities of substitution in the biological pattern,
    i.e. the matrix q in the slides.
    """
    
    q = CharMatrix(amino_acids, 0)
    total_number_substitutions = 0
    
    for a in amino_acids:
        for b in amino_acids[amino_acids_indices[a]:]:
            total_number_substitutions += f[a, b]
    
    for a in amino_acids:
        for b in amino_acids[amino_acids_indices[a]:]:
            q[a, b] = f[a, b] / total_number_substitutions
    
    return q

### Expected residue frequency
We now compute the expected frequency by residue:
$$p_a=q_{a, a} + \frac{1}{2}\sum_{a\neq b}q_{a,b}$$

In [10]:
def residue_probablities(q):
    """Computes the residue probabilities, i.e. the vector p in the slides."""
    
    p = [0] * len(amino_acids)
    
    for a in amino_acids:
        p[amino_acids_indices[a]] = q[a, a] + (sum(q[a, b] for b in amino_acids
                if a != b)) / 2
    
    return p

### Occurence probabilities in the random pattern
We now compute the occurence probability of a substitution in the random pattern:
$$
 e_{a, b} = \begin{cases}
        2p_ap_b  & \text{ when } a\neq b\\
        p_a^2 &  \text{ when } a = b
        \end{cases}
$$

In [11]:
def random_probabilities(p):
    """Computes the alignment probabilities in the random pattern, i.e. the
    matrix e in the slides.
    """
    
    e = CharMatrix(amino_acids, 0)
    
    for a in amino_acids:
        for b in amino_acids[amino_acids_indices[a]:]:
            if a == b:
                e[a, b] = p[amino_acids_indices[a]] ** 2
            else:
                e[a, b] = p[amino_acids_indices[a]] * p[amino_acids_indices[b]] * 2

    return e

### Log-odds ratios
The last step, we compute the log-odds ratios:
$$s_{a,b}=2\log_2\Big(\frac{q_{a,b}}{e_{a,b}}\Big)$$
The $s_{a,b}$ matrix is the final BLOSUM matrix.
Note that a rounding is applied on the values in order to get integers.

In [12]:
def log_odds_ratio(q, e):
    """Computes the log-odds ratio, i.e. the matrix s in the slides."""
    
    s = CharMatrix(amino_acids, 0)
    
    for a in amino_acids:
        for b in amino_acids[amino_acids_indices[a]:]:
            s[a, b] = 0 if q[a, b] == 0 else round(2 * log2(q[a, b] / e[a, b]))
    
    return s

### Making the BLOSUM matrix
The following function ties all these steps together. It accepts a domain filename and returns the BLOSUM matrix.

In [13]:
def make_blosum(domain_filename, number_blocks, required_identity, print_info = False):
    """Constructs a BLOSUM matrix from a fasta file containing a domain.
    
    Parameters:
        domain_filename: the filename of the domain to use
        number_blocks: the number of BLOCKS in the domain
        required_identity: the identity required in a cluster
    
    Return value: the BLOSUM matrix
    """
    
    if print_info:
        print("Making BLOSUM" + str(round(required_identity * 100)), "from",
                domain_filename)
    filenames = split_domain(domain_filename, number_blocks)
    frequencies = []
    
    for filename in filenames:
        if print_info:
            print("Clustering", filename, "...")
        clusters = make_clusters(filename, required_identity)
        if print_info:
            print("Computing frequencies...")
        frequencies.append(weighted_frequencies(clusters))
        
    if print_info:
        print("Normalizing BLOCKS...")
    f = normalized_sum(frequencies)
    if print_info:
        print("Computing other matrices...")
    q = biological_probabilities(f)
    p = residue_probablities(q)
    e = random_probabilities(p)
    s = log_odds_ratio(q, e)
    
    return s

### The main function
Here we generate 4 matrices, two for each domain.

In [14]:
def main():
    print(make_blosum("PDZ-domain.fasta", 2, 0.4, True))
    print(make_blosum("SH3-domain.fasta", 4, 0.4, True))
    print(make_blosum("PDZ-domain.fasta", 2, 0.7, True))
    print(make_blosum("SH3-domain.fasta", 4, 0.7, True))

if __name__ == "__main__":
    main()

  A  0 
  R  0   2 
  N  0   1   6 
  D  0   0  -1   5 
  C  2   1   0  -2  -5 
  E  1   3   0  -1   2   3 
  Q  2   1  -1   1   1   2   0 
  G  0  -3   0   0   1  -2  -1   3 
  H  2   1   1   3   0   2   4   0  -3 
  I  2   4   1   2   3   5   2   0   1   6 
  L  3   3   3   2   3   4   4   1   5   5   5 
  K  4   3   5   3   4   6   5   3   4   5   6   5 
  M  2   5   4   0   1   5   4  -1   7   4   6   3   8 
  F  4   4   0   4   4   5   5   1   6   6   6   6   6   7 
  P  4   5   2   5   1   3   6   1   6   6   6   7   7   8   9 
  S  5   1   6   3   6   3   4   6   5   4   5   8   4   7   7  10 
  T  9   9   5   9   9   7  10   6  10  11  11  13   5  12  14   9  14 
  W  3   0   2   4   9   2   4   9   8   1   5  12   6   7   8  14   6  18 
  Y  6   7   7   8   4   7   9   3  10   5  10   9  12  10  11   9  13  11  14 
  V  6   9   8   6   8  11   8   5   9  11  10   9  12  12  10  10  13  12  14  16 
     A   R   N   D   C   E   Q   G   H   I   L   K   M   F   P   S   T   W   Y  

## Results
### Method
The four generated matrices have been saved to files. These files can be found in the `output/` folder.  
The steps of the BLOSUM method have been explained above, but here is a brief summary:
* Split the domain in mutliple BLOCKS.
* Compute the weighted frequencies for each BLOCK.
* Compute the mean of the weighted frequencies between the BLOCKS.
* Compute the occurrence probabilities of substitutions in the evolution pattern.
* Compute the individual residue probabilities.
* Compute the occurrence probabilities of substitutions in the random pattern.
* Compute the log-odds ratios.

### Comparison with BLOSUM62
Here is the BLOSUM70 matrix computed for the SH3 domain:
```
  A   R   N   D   C   Q   E   G   H   I   L   K   M   F   P   S   T   W   Y   V
  4 
 -1   3 
  0   0   5 
  0   0   2   5 
  2   1   0  -1   4 
  0   3   2   2   1   4 
  1   2   2   4   1   4   5 
  1  -2   2   1   0   0   0   8 
  0   2   2   1   3   3   3   1   5 
 -2   0  -2  -3   0   0   0  -4   1   4 
 -2  -1  -1  -3   1   0  -1  -4   2   4   6 
  2   5   3   2   3   5   5   0   5   2   1   8 
  2   3   2   2   4   4   4   0   4   5   6   6   8 
 -2  -1  -2  -4   3  -1  -1  -5   3   2   3   1   5   8 
  1   0   1   1  -1   0   1   0   1  -2  -2   3   2  -3  10 
  4   4   6   5   3   5   5   3   5   1   1   6   5   1   5  11 
  5   6   6   5   5   7   7   3   7   5   5   8   8   3   4  10  11 
 -3  -3  -4  -3   3  -2  -3  -7   1   0   2  -1   5   3  -6   0   3  12 
  1   2   1  -1   6   2   1  -1   7   3   3   4   6   7  -1   4   6   6  11 
  4   4   2   0   5   4   3   1   4   7   6   6   7   6   2   5   8   2   5  11 
```
In comparison, here is the BLOSUM62 matrix without `B`, `Z` and `X` values:
```
  A   R   N   D   C   Q   E   G   H   I   L   K   M   F   P   S   T   W   Y   V
  4 
 -1   5 
 -2   0   6 
 -2  -2   1   6 
  0  -3  -3  -3   9 
 -1   1   0   0  -3   5 
 -1   0   0   2  -4   2   5 
  0  -2   0  -1  -3  -2  -2   6 
 -2   0   1  -1  -3   0   0  -2   8 
 -1  -3  -3  -3  -1  -3  -3  -4  -3   4 
 -1  -2  -3  -4  -1  -2  -3  -4  -3   2   4 
 -1   2   0  -1  -3   1   1  -2  -1  -3  -2   5 
 -1  -1  -2  -3  -1   0  -2  -3  -2   1   2  -1   5 
 -2  -3  -3  -3  -2  -3  -3  -3  -1   0   0  -3   0   6 
 -1  -2  -2  -1  -3  -1  -1  -2  -2  -3  -3  -1  -2  -4   7 
  1  -1   1   0  -1   0   0   0  -1  -2  -2   0  -1  -2  -1   4 
  0  -1   0  -1  -1  -1  -1  -2  -2  -1  -1  -1  -1  -2  -1   1   5 
 -3  -3  -4  -4  -2  -2  -3  -2  -2  -3  -2  -3  -1   1  -4  -3  -2  11 
 -2  -2  -2  -3  -2  -1  -2  -3   2  -1  -1  -2  -1   3  -3  -2  -2   2   7 
  0  -3  -3  -3  -1  -2  -2  -3  -3   3   1  -2   1  -1  -2  -2   0  -3  -1   4 
```

As we can see, a lot of substitutions that were forbidden in the original BLOSUM62 matrix are now allowed in our SH3 BLOSUM70 matrix.

BLOSUM62 have only a few positive values outside the diagonal, but in our BLOSUM70 matrix there is alot of positive values. Only the amino acids `I`, `L`, `F` and `W` have a significant amount of negative values.

The diagonals have similar values, but only a few values are equal.

### Alignment tests

I ran my project 1 with the freshly created BLOSUM70 matrix, and with two SH3 sequences (with global alignment):
```
1 solution were found.
                    10        20        30        40        50        60 
Sequence 1: GGVTTFVALYDYESRTETDLSFKKGERLQIVNNTEGDWWLAHSLSTGQTGYIPSNYVAPSDS
            ......::::::......::...::....:.......::.:.. ..::.::::::::.....
Sequence 2: SELKKVVALYDYMPMNANDLQLRKGDEYFILEESNLPWWRARD-KNGQEGYIPSNYVTEAED

```
And with the BLOSUM40 matrix:
```
1 solution were found.
                    10        20        30        40        50        60 
Sequence 1: GGVTTFVALYDYESRTETDLSFKKGERLQIVNNTEGDWWLAHSLSTGQTGYIPSNYVAPSDS
            ......::::::......::...::....:.......::.:.. ..::.::::::::.....
Sequence 2: SELKKVVALYDYMPMNANDLQLRKGDEYFILEESNLPWWRARD-KNGQEGYIPSNYVTEAED

```
I also have done tests on other SH3 sequences, the two BLOSUM matrices always gave the same results. That is probably due to the fact that the SH3 sequences are very short and similar.

Alignment for the same sequences, but with the original BLOSUM62 matrix:
```
1 solution were found.
                    10        20        30        40        50        60 
Sequence 1: GGVTTFVALYDYESRTETDLSFKKGERLQIVNNTEGDWWLAHSLSTGQTGYIPSNYVAPSDS
            ......::::::......::...::....:.......::.:.. ..::.::::::::.....
Sequence 2: SELKKVVALYDYMPMNANDLQLRKGDEYFILEESNLPWWRARD-KNGQEGYIPSNYVTEAED
                    10        20        30        40         50        60
```
This alignment is identical to the one given by our new matrices, for the same reason as in the previous paragraph.

However, the alignments given by our BLOSUM SH3 matrices should be better for aligning arbitrary SH3 sequences than the generic BLOSUM62, because the values we computed came from SH3 alignments only. Thus they reflects better the substitution probabilities than the generic BLOSUM, because this latter one is also influenced by other domains.